In [1]:
from selenium import webdriver


In [6]:
options = webdriver.ChromeOptions()
options.add_argument('ignore-certificate-errors')
driver = webdriver.Chrome(chrome_options=options)

<ipython-input-6-62b4b67480f1>:3: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options)
